In [1]:
%pip install keras_self_attention
%pip install keras-tuner
%pip install gpflow

  Created wheel for keras-self-attention: filename=keras_self_attention-0.49.0-cp37-none-any.whl size=19468 sha256=6101b26a6d9f00f386006b8c1e2a8cbe8acc27abdb8f87160f2f54acc1b42b1e
  Stored in directory: /root/.cache/pip/wheels/6f/9d/c5/26693a5092d9313daeae94db04818fc0a2b7a48ea381989f34
Successfully built keras-self-attention
     |████████████████████████████████| 71kB 6.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=1de2525ac18384ff051d226c21d1963dd09462e6a30e1c3cb245e3c74ea4c6d9
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=ca005cf041ccc7bfad7b2a53e802ec699b59b85238805c1ecbec13556a32c51d
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables
     |█████████████████████████

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import BatchNormalization, Dense, LeakyReLU, Dropout
from keras_self_attention import SeqSelfAttention, SeqWeightedAttention
import site
import pandas as pd
import numpy as np
import matplotlib
import os

In [3]:
pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 40)

In [4]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import h5py as h5
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import errno
import os
from collections import defaultdict
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.summary import create_file_writer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import kerastuner as kt
from kerastuner import HyperModel
import numpy as np
import itertools
import multiprocessing
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import BatchNormalization, Dense, LeakyReLU, Dropout
from keras_self_attention import SeqSelfAttention, SeqWeightedAttention
import site
import pandas as pd
import numpy as np
import matplotlib
import os

In [5]:
def get_features_model(model_name, combined_models):
    for ids, models in combined_models:
        if models[0].name == model_name:
            return models[1]
    return None

def get_features_from_combined_models(combined_models, X_input):
    data_features = []
    for ids, models in combined_models:
        data_features.append(np.array(models[1].predict(X_input), dtype='float64'))
    return np.concatenate(tuple(data_features), axis=1)


In [6]:
def coShuffled_vectors(X, Y):
    if tf.shape(X)[0] == tf.shape(Y)[0]:
        test_idxs = tf.range(start=0, limit=tf.shape(X)[0], dtype=tf.int32)
        shuffled_test_idxs = tf.random.shuffle(test_idxs)
        return (tf.gather(X, shuffled_test_idxs), tf.gather(Y, shuffled_test_idxs))
    else:
        raise ValueError(f"0-dimension has to be the same {tf.shape(X)[0]} != {tf.shape(Y)[0]}")


def getNpArrayFromH5(hf_Data):
    X_train = hf_Data['Train_Data']  # Get train set
    X_train = np.array(X_train)
    Y_train = hf_Data['Label']  # Get train label
    Y_train = np.array(Y_train)
    return X_train, Y_train

# data extraction
def getData(is500=True, shuffle=False, ise2e=False, include_secondary=False, validation_split=None, isColab=False):
    if not include_secondary:
        hf_Train = h5.File(
            f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/{"e2e_Train_Data" if ise2e else "Fold_10_Train_Data"}_{str(500) if is500 else str(1000)}.h5', 'r')
        hf_Test = h5.File(
            f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/{"e2e_Test_Data" if ise2e else "Fold_10_Test_Data"}_{str(500) if is500 else str(1000)}.h5', 'r')
    else:
        hf_Train = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Train_Secondary_Data_1136.h5', 'r')
        hf_Test = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Test_Secondary_Data_1136.h5', 'r')

    X_train, Y_train = getNpArrayFromH5(hf_Train)
    X_test, Y_test = getNpArrayFromH5(hf_Test)
    Y_train = to_categorical(Y_train, 13)  # Process the label of tain
    Y_test = to_categorical(Y_test, 13)  # Process the label of te

    if shuffle:
        X_train, Y_train = coShuffled_vectors(X_train, Y_train)
        X_test, Y_test = coShuffled_vectors(X_test, Y_test)

    X_validation = Y_validation = None
    if validation_split is not None:
        # sklearn split shuffles anyway
        X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=validation_split)

    return X_train, Y_train, X_test, Y_test, X_validation, Y_validation


def getE2eData(is500=True, shuffle=False, include_secondary=False, isColab=False):
    if not include_secondary:
        hf_Train = h5.File(
            f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Train_Data_{str(500) if is500 else str(1000)}.h5', 'r')
        hf_Test = h5.File(
            f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Test_Data_{str(500) if is500 else str(1000)}.h5', 'r')
    else:
        hf_Train = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Train_Secondary_Data_1136.h5', 'r')
        hf_Test = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Test_Secondary_Data_1136.h5', 'r')

    X_train, Y_train = getNpArrayFromH5(hf_Train)
    X_test, Y_test = getNpArrayFromH5(hf_Test)
    Y_train = to_categorical(Y_train, 13)  # Process the label of tain
    Y_test = to_categorical(Y_test, 13)  # Process the label of te

    if shuffle:
        X_train, Y_train = coShuffled_vectors(X_train, Y_train)
        X_test, Y_test = coShuffled_vectors(X_test, Y_test)

    hf_Val = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Val_Secondary_Data_1136.h5', 'r') if include_secondary else h5.File(
        f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Val_Data_{str(500) if is500 else str(1000)}.h5', 'r')
    X_validation, Y_validation = getNpArrayFromH5(hf_Val)
    Y_validation = to_categorical(Y_validation, 13)  # Process the label of tain

    return X_train, Y_train, X_test, Y_test, X_validation, Y_validation


def getE2eDataJustSecondary(shuffle=False,isColab=False):
    hf_Train = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Train_just_Secondary_Data_1000.h5', 'r')
    hf_Test = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Test_just_Secondary_Data_1000.h5', 'r')

    X_train, Y_train = getNpArrayFromH5(hf_Train)
    X_test, Y_test = getNpArrayFromH5(hf_Test)
    Y_train = to_categorical(Y_train, 13)  # Process the label of tain
    Y_test = to_categorical(Y_test, 13)  # Process the label of te

    if shuffle:
        X_train, Y_train = coShuffled_vectors(X_train, Y_train)
        X_test, Y_test = coShuffled_vectors(X_test, Y_test)

    hf_Val = h5.File(f'./{"data" if not isColab else "drive/MyDrive/data_papers/ncRNA"}/e2e_Val_just_Secondary_Data_1000.h5', 'r')
    
    X_validation, Y_validation = getNpArrayFromH5(hf_Val)
    Y_validation = to_categorical(Y_validation, 13)  # Process the label of tain

    return X_train, Y_train, X_test, Y_test, X_validation, Y_validation


In [7]:
def reverse_one_hot(Y_input):
    return np.apply_along_axis(np.argmax, 1, Y_input) + 1
    
def sparse_setdiff(a1, a2):
    a1a = a1.reshape(a1.shape[0], -1)
    a2a = a2.reshape(a2.shape[0], -1)
    spa2a = [np.where(x)[0].tolist() for x in a2a]
    spa1a = [np.where(x)[0].tolist() for x in a1a]
    idxs_to_keep = []
    for idx, sample in enumerate(spa1a):
        try:
            spa2a.index(sample)
        except ValueError:
            # not in list
            idxs_to_keep.append(idx)
    return a1[idxs_to_keep], idxs_to_keep

def get_combined_features_from_models(
    
        to_combine,
        X_train, Y_train,
        X_test, Y_test,
        reverse_one_hot=False,
        normalize_X_func=None):
    
    models = []
    models_dict = {}
    X_trains_out = []
    X_test_out = []
    XY_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))

    models_have_different_inputs = isinstance(Y_train,list)

    if reverse_one_hot:
        if models_have_different_inputs:
            Y_train_new = np.apply_along_axis(np.argmax, 1, Y_train) + 1
            Y_test_new = np.apply_along_axis(np.argmax, 1, Y_test) + 1
        else:
            Y_train_new = [ np.apply_along_axis(np.argmax, 1, y_train) + 1 for y_train in Y_train ]  
            Y_test_new = [ np.apply_along_axis(np.argmax, 1, y_test) + 1 for y_test in Y_test ]              
    else:
        if models_have_different_inputs:
            Y_train_new = Y_train.copy()
            Y_test_new = Y_test.copy()
        else:
            Y_train_new = [ y_train.copy() for y_train in Y_train ] 
            Y_test_new = [ y_test.copy() for y_test in Y_train ] 
            

    extraction_counter =0
    for model_file_name, layer_name, kwargs in to_combine:
        model_here = None
        if isinstance(model_file_name, tf.keras.models.Model):
            model_here = model_file_name
            model_file_name = model_here.name
        else:
            if model_file_name in models_dict.keys():
                model_here = models_dict[model_file_name]
            else:
                model_here = tf.keras.models.load_model(model_file_name,
                                                        **kwargs) if kwargs is not None else tf.keras.models.load_model \
                    (model_file_name)

        features_model = Model(model_here.input,
                               get_layer_by_name(model_here.layers, layer_name).output)
        
        if normalize_X_func is None:
            X_trains_out.append(np.array(features_model.predict(X_train if not models_have_different_inputs else X_train[extraction_counter]), dtype='float64'))
            X_test_out.append(np.array(features_model.predict(X_test if not models_have_different_inputs else X_test[extraction_counter]), dtype='float64'))
        else:
            X_trains_out.append(np.array(normalize_X_func(features_model.predict(X_train if not models_have_different_inputs else X_train[extraction_counter])), dtype='float64'))
            X_test_out.append(np.array(normalize_X_func(features_model.predict(X_test if not models_have_different_inputs else X_test[extraction_counter])), dtype='float64'))
        XY_dict[model_file_name][layer_name]['Train']['X'] = X_trains_out[-1]
        XY_dict[model_file_name][layer_name]['Test']['X'] = X_test_out[-1]
        XY_dict[model_file_name][layer_name]['Train']['Y'] = Y_train_new
        XY_dict[model_file_name][layer_name]['Test']['Y'] = Y_test_new
        models.append(((model_file_name, layer_name), (model_here, features_model)))
        models_dict[model_file_name] = model_here
        extraction_counter += 1

    X_train_new = np.concatenate(tuple(X_trains_out), axis=1)
    X_test_new = np.concatenate(tuple(X_test_out), axis=1)

    data_train = (X_train_new, Y_train_new)
    data_test = (X_test_new, Y_test_new)

    return models, data_train, data_test, XY_dict

def get_layer_by_name(layers, name, return_first=True):
    matching_named_layers = [l for l in layers if l.name == name]
    if not matching_named_layers:
        return None
    return matching_named_layers[0] if return_first else matching_named_layers


def make_dir_if_not_exist(used_path):
    if not os.path.isdir(used_path):
        try:
            os.mkdir(used_path)
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise exc
            else:
                raise ValueError(f'{used_path} directoy cannot be created because its parent directory does not exist.')


def source_model(model_func, model_name, input_shape):
    m = None
    if isinstance(model_func, tf.keras.models.Model):
        m = model_func
        m._name = model_name
    else:
        m = model_func(model_name, input_shape)
    return m

In [8]:
import gpflow
from gpflow.utilities import ops, print_summary, set_trainable
from gpflow.config import set_default_float, default_float, set_default_summary_fmt
from gpflow.ci_utils import ci_niter
import random


def get_gp_acc(mGP, data_gp):
    (Y_test_GP_mean, Y_test_GP_variance) = mGP.predict_y(data_gp[0])
    Y_test_GP_mean = Y_test_GP_mean.numpy()
    Y_test_pred = []
    for i in range(data_gp[0].shape[0]):
        Y_test_pred.append(np.argmax(Y_test_GP_mean[i,]))
    diffs = Y_test_pred - data_gp[1]
    test_acc = sum(1 for x in diffs if x == 0) / data_gp[0].shape[0]
    return test_acc, Y_test_pred, (Y_test_GP_mean, Y_test_GP_variance)

def get_features_model(model_name, combined_models):
    for ids, models in combined_models:
        if models[0].name == model_name:
            return models[1]
    return None

def get_features_from_combined_models(combined_models, X_input):
    data_features = []
    for ids, models in combined_models:
        data_features.append(np.array(models[1].predict(X_input), dtype='float64'))
    return np.concatenate(tuple(data_features), axis=1)

In [9]:
# 'new' data 
X_train_1000e, Y_train_1000e, X_test_1000e, Y_test_1000e, X_val_1000e, Y_val_1000e = getE2eData(is500=False,
                                                                                                    include_secondary=False,
                                                                                                    isColab=True)
X_train_1000e_w2nd, Y_train_1000e_w2nd, X_test_1000e_w2nd, Y_test_1000e_w2nd, X_val_1000e_w2nd, Y_val_1000e_w2nd = getE2eData(is500=False, include_secondary=True, isColab=True)
X_train_1000e_j2nd, Y_train_1000e_j2nd, X_test_1000e_j2nd, Y_test_1000e_j2nd, X_val_1000e_j2nd, Y_val_1000e_j2nd = getE2eDataJustSecondary(isColab=True)

In [10]:
print (X_train_1000e.shape)
print (X_train_1000e_w2nd.shape)
print (X_train_1000e_j2nd.shape)

(6858, 1000, 8)
(6858, 1136, 12)
(6858, 1000, 3)


In [11]:
# merge into a new train:
X_new_train = np.concatenate( (X_train_1000e, X_val_1000e), axis=0 )
Y_new_train = np.concatenate( (Y_train_1000e, Y_val_1000e), axis=0 )    

X_new_train_j2nd = np.concatenate( (X_train_1000e_j2nd, X_val_1000e_j2nd), axis=0 )
Y_new_train_j2nd = np.concatenate( (Y_train_1000e_j2nd, Y_val_1000e_j2nd), axis=0 )    

X_new_train_w2nd = np.concatenate( (X_train_1000e_w2nd, X_val_1000e_w2nd), axis=0 )
Y_new_train_w2nd = np.concatenate( (Y_train_1000e_w2nd, Y_val_1000e_w2nd), axis=0 )    


In [12]:
# CNNs no secondary
mCNN1_1000 = load_model("./drive/MyDrive/data_papers/ncRNA/CNN_baseline_May16_e2e1000_256.h5") # CNN on 256 1st dens
mCNN1_1000._name = "cnn_merged_newdata_finalist_1"

mCNN2_1000 = load_model("./drive/MyDrive/data_papers/ncRNA/CNN_baseline_May16_e2e.h5")   # CNN on 128 1st dens
mCNN2_1000._name = "cnn_merged_newdata_finalist_2"

mCNN_1000 = load_model("./drive/MyDrive/data_papers/ncRNA/cnn_noTest_20210516_model_445_0.998")   # CNN on 128 1st dens
mCNN_1000._name = "cnn_merged_newdata_colab_finalist"

mCNN_1000 = load_model("./drive/MyDrive/data_papers/ncRNA/cnn_noTest_20210516_model_445_0.998")   # CNN on 128 1st dens
mCNN_1000._name = "cnn_merged_newdata_colab_finalist"

# CNN w/ secondary
mCNN_1000_w2nd = load_model("./drive/MyDrive/data_papers/ncRNA/CNN_baseline_May16_e2e_secondary.h5", custom_objects=SeqWeightedAttention.get_custom_objects())
mCNN_1000_w2nd._name = "cnn_merged_newdata_w_secondary_finalist"

# CNN secondary only
mCNN_1000_j2nd = load_model("./drive/MyDrive/data_papers/ncRNA/cnn_j2nd_noTest_20210516_model_488_0.990", custom_objects=SeqWeightedAttention.get_custom_objects())
mCNN_1000_j2nd._name = "cnn_merged_newdata_j_secondary_finalist"

# RNN on the same data
RNN_1000 = load_model("./drive/MyDrive/data_papers/ncRNA/RNN_baseline_17May_180.h5", custom_objects=SeqWeightedAttention.get_custom_objects())   # RNN on 180ep 1000 ts
RNN_1000._name = "rnn_merged_newdata_colab_finalist"

# RNN with secondary still needs to continue fit ## requires the validation in the fit data!
RNN_1000_w2nd = load_model("./drive/MyDrive/data_papers/ncRNA/rnn_newdata_w2nd_colab_continue_fit_epoch_22_accuracy_0.959.h5", custom_objects=SeqWeightedAttention.get_custom_objects())   # RNN on 180ep 1000 ts
RNN_1000_w2nd._name = "rnn_newdata_w2nd_colab_continue_fit"

# RNN on the e2e data just secondary
RNN_1000_j2nd = load_model("./drive/MyDrive/data_papers/ncRNA/rnn_newdata_j2nd_colab_continue_fit.h5", custom_objects=SeqWeightedAttention.get_custom_objects())   # RNN on 180ep 1000 ts
RNN_1000_j2nd._name = "rnn_newdata_j2nd_colab_continue_fit"


mCNN1_1000.evaluate(X_test_1000e, Y_test_1000e)  # 96.15% 
mCNN2_1000.evaluate(X_test_1000e, Y_test_1000e)  # 95.80 %  
mCNN_1000.evaluate(X_test_1000e, Y_test_1000e)  # 95.57% 
mCNN_1000_w2nd.evaluate(X_test_1000e_w2nd, Y_test_1000e_w2nd)  # 94.64 %
mCNN_1000_j2nd.evaluate(X_test_1000e_j2nd, Y_test_1000e_j2nd)  # 72.96 %
RNN_1000.evaluate(X_test_1000e, Y_test_1000e)  # 95.45 %
RNN_1000_w2nd.evaluate(X_test_1000e_w2nd, Y_test_1000e_w2nd)  # 91.72 %
RNN_1000_j2nd.evaluate(X_test_1000e_j2nd, Y_test_1000e_j2nd)  # 63.52 %

27/27 [==============================] - 32s 1s/step - loss: 1.3527 - accuracy: 0.6352


[1.3526557683944702, 0.6351981163024902]

In [ ]:
data_train_ll_no2nd_j2nd[0].shape

In [13]:
    to_combine_last_layers_no2nd_j2nd = [
        (mCNN1_1000, "dense_26", None),
        (mCNN2_1000, "dense_14", None),
        (mCNN_1000_j2nd,"dense_5", None)
    ]

    combined_models_no2nd_j2nd, data_train_ll_no2nd_j2nd, data_test_ll_no2nd_j2nd, data_access_ll_no2nd_j2nd = get_combined_features_from_models(
        to_combine_last_layers_no2nd_j2nd,
        [ X_new_train, X_new_train, X_new_train_j2nd],
        [ Y_new_train, Y_new_train, Y_new_train_j2nd], 
        [ X_test_1000e, X_test_1000e, X_test_1000e_j2nd],
        [ Y_test_1000e, Y_test_1000e, Y_test_1000e_j2nd],
        reverse_one_hot=False)

    to_combine_last_layers_no2nd_j2nd_rNo2nd = [
        (mCNN1_1000, "dense_26", None),
        (mCNN_1000_j2nd,"dense_5", None),
        (RNN_1000,"dense_3", None)
    ]

    combined_models_no2nd_j2nd_rNo2nd, data_train_ll_no2nd_j2nd_rNo2nd, data_test_ll_no2nd_j2nd_rNo2nd, data_access_ll_no2nd_j2nd_rNo2nd = get_combined_features_from_models(
        to_combine_last_layers_no2nd_j2nd_rNo2nd,
        [ X_new_train, X_new_train_j2nd, X_new_train],
        [ Y_new_train, Y_new_train_j2nd, Y_new_train], 
        [ X_test_1000e, X_test_1000e_j2nd, X_test_1000e],
        [ Y_test_1000e, Y_test_1000e_j2nd, Y_test_1000e],
        reverse_one_hot=False)




In [14]:
    to_combine_penul_layers_no2nd_j2nd = [
        (mCNN1_1000, "dense_25", None),
        (mCNN2_1000, "dense_13", None),
        (mCNN_1000_j2nd,"dense_4", None)
    ]

    combined_models_penul_no2nd_j2nd, data_train_ll_penul_no2nd_j2nd, data_test_ll_penul_no2nd_j2nd, data_access_ll_penul_no2nd_j2nd = get_combined_features_from_models(
        to_combine_penul_layers_no2nd_j2nd,
        [ X_new_train, X_new_train, X_new_train_j2nd],
        [ Y_new_train, Y_new_train, Y_new_train_j2nd], 
        [ X_test_1000e, X_test_1000e, X_test_1000e_j2nd],
        [ Y_test_1000e, Y_test_1000e, Y_test_1000e_j2nd],
        reverse_one_hot=False)

    to_combine_penul_layers_no2nd_j2nd_rNo2nd = [
        (mCNN1_1000, "dense_25", None),
        (mCNN_1000_j2nd,"dense_4", None),
        (RNN_1000,"dense_2", None)
    ]

    combined_models_penul_no2nd_j2nd_rNo2nd, data_train_ll_penul_no2nd_j2nd_rNo2nd, data_test_ll_penul_no2nd_j2nd_rNo2nd, data_access_ll_penul_no2nd_j2nd_rNo2nd = get_combined_features_from_models(
        to_combine_penul_layers_no2nd_j2nd_rNo2nd,
        [ X_new_train, X_new_train_j2nd, X_new_train],
        [ Y_new_train, Y_new_train_j2nd, Y_new_train], 
        [ X_test_1000e, X_test_1000e_j2nd, X_test_1000e],
        [ Y_test_1000e, Y_test_1000e_j2nd, Y_test_1000e],
        reverse_one_hot=False)

In [15]:
    to_combine_last_layers_no2nd_rNo2nd = [
        (mCNN1_1000, "dense_26", None),
        (RNN_1000,"dense_3", None)
    ]

    combined_models_no2nd_rNo2nd, data_train_ll_no2nd_rNo2nd, data_test_ll_no2nd_rNo2nd, data_access_ll_no2nd_rNo2nd = get_combined_features_from_models(
        to_combine_last_layers_no2nd_rNo2nd,
        [ X_new_train, X_new_train],
        [ Y_new_train,  Y_new_train], 
        [ X_test_1000e,  X_test_1000e],
        [ Y_test_1000e,  Y_test_1000e],
        reverse_one_hot=False)
    
    to_combine_penul_layers_no2nd_rNo2nd = [
        (mCNN1_1000, "dense_25", None),
        (RNN_1000,"dense_2", None)
    ]

    combined_models_no2nd_rNo2nd_penul, data_train_penul_no2nd_rNo2nd, data_test_penul_no2nd_rNo2nd, data_access_penul_no2nd_rNo2nd = get_combined_features_from_models(
        to_combine_penul_layers_no2nd_rNo2nd,
        [ X_new_train, X_new_train],
        [ Y_new_train,  Y_new_train], 
        [ X_test_1000e,  X_test_1000e],
        [ Y_test_1000e,  Y_test_1000e],
        reverse_one_hot=False)


In [16]:
    to_combine_last2_layers_no2nd_rNo2nd = [
        (mCNN1_1000, "dense_25", None),
        (mCNN1_1000, "dense_26", None),
        (RNN_1000,"dense_2", None),
        (RNN_1000,"dense_3", None)
    ]

    combined_models_last2_no2nd_rNo2nd, data_train_last2_no2nd_rNo2nd, data_test_last2_no2nd_rNo2nd, data_access_last2_no2nd_rNo2nd = get_combined_features_from_models(
        to_combine_last2_layers_no2nd_rNo2nd,
        [ X_new_train, X_new_train, X_new_train, X_new_train],
        [ Y_new_train,  Y_new_train, Y_new_train,  Y_new_train], 
        [ X_test_1000e,  X_test_1000e, X_test_1000e,  X_test_1000e],
        [ Y_test_1000e,  Y_test_1000e, Y_test_1000e,  Y_test_1000e],
        reverse_one_hot=False)
    
    to_combine_last2_layers_no2nd_j2nd_rNo2nd = [
        (mCNN1_1000, "dense_25", None),
        (mCNN1_1000, "dense_26", None),
        (mCNN_1000_j2nd,"dense_4", None),
        (mCNN_1000_j2nd,"dense_5", None),
        (RNN_1000,"dense_2", None),
        (RNN_1000,"dense_3", None)
    ]

    combined_models_last2_no2nd_j2nd_rNo2nd, data_train_last2_no2nd_j2nd_rNo2nd, data_test_last2_no2nd_j2nd_rNo2nd, data_access_last2_no2nd_j2nd_rNo2nd = get_combined_features_from_models(
        to_combine_last2_layers_no2nd_j2nd_rNo2nd,
        [ X_new_train, X_new_train, X_new_train_j2nd, X_new_train_j2nd, X_new_train, X_new_train],
        [ Y_new_train,  Y_new_train, Y_new_train_j2nd, Y_new_train_j2nd, Y_new_train, Y_new_train], 
        [ X_test_1000e,  X_test_1000e, X_test_1000e_j2nd, X_test_1000e_j2nd, X_test_1000e, X_test_1000e],
        [ Y_test_1000e,  Y_test_1000e, Y_test_1000e_j2nd, Y_test_1000e_j2nd, Y_test_1000e, Y_test_1000e],
        reverse_one_hot=False)

In [18]:
def get_gp_oos(data_gp, data_test, lengthscales_input, white_variance, kernel_func, num_inducing=100, num_it=10000):
  try:
    lengthscales = tf.convert_to_tensor([lengthscales_input] * data_gp[0].shape[1], dtype=default_float())
    kernel = (kernel_func_map[kernel_func](lengthscales=lengthscales)  + gpflow.kernels.White(variance=white_variance)) if white_variance > 0.0 else kernel_func_map[kernel_func](lengthscales=lengthscales)
    invlink = gpflow.likelihoods.RobustMax(np.max(data_test[1])+1)  # Robustmax inverse link function
    likelihood = gpflow.likelihoods.MultiClass(np.max(data_test[1])+1, invlink=invlink)  # Multiclass likelihood
    M = num_inducing # Number of inducing locations
    Z = data_gp[0][sorted(random.sample(range(0,data_gp[0].shape[0]),M)),:].copy()
    mGP = gpflow.models.SVGP(
        kernel=kernel,
        likelihood=likelihood,
        inducing_variable=Z,
        num_latent_gps=np.max(data_test[1])+1,
        whiten=True,
        q_diag=False,
    )
    # Only train the variational parameters
    set_trainable(mGP.kernel.kernels[1].variance, True)
    set_trainable(mGP.inducing_variable, True)
    opt = gpflow.optimizers.Scipy()
    opt_logs = opt.minimize(
        mGP.training_loss_closure(data_gp), mGP.trainable_variables,
        options=dict(maxiter=ci_niter(num_it))
    )
    test_acc, Y_test_pred, (gp_mean, gp_var) = get_gp_acc(mGP, data_test)
    print(test_acc, lengthscales_input, white_variance, kernel_func_map[kernel_func], num_inducing)  
    return test_acc, lengthscales_input, white_variance, kernel_func, num_inducing, Y_test_pred, mGP
  except :
     return -1.0, lengthscales_input, white_variance, kernel_func, num_inducing, None, mGP


In [19]:
# reproducibility:
np.random.seed(0)
tf.random.set_seed(123)

In [20]:
kernel_func_map = dict(zip(list(range(7)), 
                           [gpflow.kernels.Cosine, 
                            gpflow.kernels.SquaredExponential, 
                            gpflow.kernels.Exponential, 
                            gpflow.kernels.Matern12, 
                            gpflow.kernels.Matern32, 
                            gpflow.kernels.Matern52, 
                            gpflow.kernels.RationalQuadratic]))  

In [21]:
input_sets = { 'lengthscales': [1e-2,1e-1,1,10.0,100.0], 
                'white_variance' : [ 1e-3, 1e-2, 1e-1, 1.0, 10.0],
                   'kernel_funcs': list(range(7)),
              'num_inducing' : [20,40,60,80,120,200]
                   }
combs = np.stack(np.meshgrid(input_sets['lengthscales'],input_sets['white_variance'],input_sets['kernel_funcs'],input_sets['num_inducing']),-1).reshape(-1,len(input_sets.keys()))

In [ ]:
# running the whole combination set
# results = np.apply_along_axis(lambda x: get_gp_oos(x[0],x[1],int(x[2]), int(x[3])) , 1, combs) 

In [22]:
# just running one
data_gp_train = (data_train_ll_no2nd_j2nd[0], reverse_one_hot(data_train_ll_no2nd_j2nd[1][0])-1)
data_gp_test = (data_test_ll_no2nd_j2nd[0], reverse_one_hot(data_test_ll_no2nd_j2nd[1][0])-1)
resultsA = [ get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 250, 4000),  get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 100, 4000) ] 

data_gp_train = (data_train_ll_no2nd_j2nd_rNo2nd[0], reverse_one_hot(data_train_ll_no2nd_j2nd_rNo2nd[1][0])-1)
data_gp_test = (data_test_ll_no2nd_j2nd_rNo2nd[0], reverse_one_hot(data_test_ll_no2nd_j2nd_rNo2nd[1][0])-1)                                                                                                    
resultsB = [ get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 250, 4000),  get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 100, 4000) ] 

0.9522144522144522 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9615384615384616 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 100
0.9603729603729604 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9615384615384616 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 100


In [ ]:
# just running one repeatedly
data_gp_train = (data_train_ll_no2nd_j2nd[0], reverse_one_hot(data_train_ll_no2nd_j2nd[1][0])-1)
data_gp_test = (data_test_ll_no2nd_j2nd[0], reverse_one_hot(data_test_ll_no2nd_j2nd[1][0])-1)
resultsGroupA = [ get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 250, 4000) for i in range(100) ]

oos_fn = "/content/gp_ll_no2nd256_no2nd128_j2nd256.csv"
gp_no2nd256_no2nd128_j2nd256_oos_acc = np.array([x[0] for x in resultsGroupA])
gp_no2nd256_no2nd128_j2nd256_oos_acc.tofile(oos_fn, sep=",")

from google.colab import files
files.download(oos_fn)

0.958041958041958 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9615384615384616 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9627039627039627 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9638694638694638 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.951048951048951 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9638694638694638 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9568764568764568 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9638694638694638 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9557109557109557 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9603729603729604 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9557109557109557 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9592074592074592 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9568764568764568 0.1 0.01 <class 'gpflow.kernels.sta

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def get_gp_oos_minmaxscaler(data_gp, data_test, lengthscales_target_avg, maxDev_ratio, kernel_func, num_inducing=100, num_it=4000):
  try:
    maxDev = lengthscales_target_avg*maxDev_ratio
    sd_train_features = np.apply_along_axis(lambda x: np.std(x)**2, 0, data_gp_train[0])
    scalerMM = MinMaxScaler()
    scaledMM = scalerMM.fit_transform(sd_train_features.reshape(-1,1)).reshape(1,-1)
    lengthscales = lengthscales_target_avg + ( scaledMM - 0.5 )*maxDev

    lengthscales = tf.convert_to_tensor(lengthscales, dtype=default_float())
    kernel = kernel_func_map[kernel_func](lengthscales=lengthscales)
    invlink = gpflow.likelihoods.RobustMax(np.max(data_test[1])+1)  # Robustmax inverse link function
    likelihood = gpflow.likelihoods.MultiClass(np.max(data_test[1])+1, invlink=invlink)  # Multiclass likelihood
    M = num_inducing # Number of inducing locations
    Z = data_gp[0][sorted(random.sample(range(0,data_gp[0].shape[0]),M)),:].copy()
    mGP = gpflow.models.SVGP(
        kernel=kernel,
        likelihood=likelihood,
        inducing_variable=Z,
        num_latent_gps=np.max(data_test[1])+1,
        whiten=True,
        q_diag=False,
    )
    # Only train the variational parameters
    # set_trainable(mGP.kernel.kernels[1].variance, True)
    set_trainable(mGP.inducing_variable, False)
    opt = gpflow.optimizers.Scipy()
    opt_logs = opt.minimize(
        mGP.training_loss_closure(data_gp), mGP.trainable_variables,
        options=dict(maxiter=ci_niter(num_it))
    )
    test_acc, Y_test_pred, (gp_mean, gp_var) = get_gp_acc(mGP, data_test)
    print(test_acc, lengthscales_target_avg, maxDev_ratio, kernel_func_map[kernel_func], num_inducing)  
    return test_acc, lengthscales_target_avg, maxDev_ratio, kernel_func, num_inducing, Y_test_pred, mGP
  except :
     return -1.0, lengthscales_target_avg, maxDev_ratio, kernel_func, num_inducing, None, None


In [ ]:
data_gp_train = (data_train_ll_no2nd_j2nd_rNo2nd[0], reverse_one_hot(data_train_ll_no2nd_j2nd_rNo2nd[1][0])-1)
data_gp_test = (data_test_ll_no2nd_j2nd_rNo2nd[0], reverse_one_hot(data_test_ll_no2nd_j2nd_rNo2nd[1][0])-1) 

In [ ]:
# input_sets_varLengthscale = { 'lengthscales': [0.05,0.1,0.2,1.0], 'max_devs' : [ 0.1, 0.25, 0.5, 0.9], 'kernel_funcs': [1,2,3,4,5,6],'num_inducing' : [200] }
input_sets_varLengthscale = { 'lengthscales': [ x*0.05 + 0.25 for x in range(12)], 'max_devs' : [ x*0.05 + 0.05 for x in range(18) ], 'kernel_funcs': [6],'num_inducing' : [200] }
combs_varLengthscale = np.stack(np.meshgrid(input_sets_varLengthscale['lengthscales'],
                                            input_sets_varLengthscale['max_devs'],
                                            input_sets_varLengthscale['kernel_funcs'],
                                            input_sets_varLengthscale['num_inducing']),-1).reshape(-1,len(input_sets_varLengthscale.keys()))

In [ ]:
# running the whole combination set
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
results_minMaxScaler_no2nd_j2nd_rNo2nd = np.apply_along_axis(lambda x: get_gp_oos_minmaxscaler(data_gp_train, data_gp_test, x[0],x[1],int(x[2]), int(x[3])) , 1, combs_varLengthscale) 

0.9592074592074592 0.25 0.05 <class 'gpflow.kernels.stationaries.RationalQuadratic'> 200
0.9615384615384616 0.3 0.05 <class 'gpflow.kernels.stationaries.RationalQuadratic'> 200
0.9533799533799534 0.35 0.05 <class 'gpflow.kernels.stationaries.RationalQuadratic'> 200
0.9615384615384616 0.4 0.05 <class 'gpflow.kernels.stationaries.RationalQuadratic'> 200
0.9487179487179487 0.45 0.05 <class 'gpflow.kernels.stationaries.RationalQuadratic'> 200
0.9557109557109557 0.5 0.05 <class 'gpflow.kernels.stationaries.RationalQuadratic'> 200
0.9533799533799534 0.55 0.05 <class 'gpflow.kernels.stationaries.RationalQuadratic'> 200
0.9440559440559441 0.6000000000000001 0.05 <class 'gpflow.kernels.stationaries.RationalQuadratic'> 200
0.9324009324009324 0.65 0.05 <class 'gpflow.kernels.stationaries.RationalQuadratic'> 200
0.9568764568764568 0.7 0.05 <class 'gpflow.kernels.stationaries.RationalQuadratic'> 200
0.9370629370629371 0.75 0.05 <class 'gpflow.kernels.stationaries.RationalQuadratic'> 200
0.947552447

In [ ]:
combs_varLengthscale

array([[1.0e-01, 1.0e-01, 4.0e+00, 2.0e+02],
       [2.0e-01, 1.0e-01, 4.0e+00, 2.0e+02],
       [1.0e-01, 2.5e-01, 4.0e+00, 2.0e+02],
       [2.0e-01, 2.5e-01, 4.0e+00, 2.0e+02],
       [1.0e-01, 5.0e-01, 4.0e+00, 2.0e+02],
       [2.0e-01, 5.0e-01, 4.0e+00, 2.0e+02],
       [1.0e-01, 9.0e-01, 4.0e+00, 2.0e+02],
       [2.0e-01, 9.0e-01, 4.0e+00, 2.0e+02]])

In [ ]:
# just running one repeatedly

resultsGroupA = [ get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 250, 4000) for i in range(400) ]
oos_fn = "/content/gp_no2nd256_no2nd128_j2nd256.csv"
gp_no2nd256_no2nd128_j2nd256_oos_acc = np.array([x[0] for x in resultsGroupA])
gp_no2nd256_no2nd128_j2nd256_oos_acc.tofile(oos_fn, sep=",")
from google.colab import files
files.download(oos_fn)

In [ ]:
## strange behaviour for penultimate layers (ignore for now)


data_gp_train = (data_train_ll_penul_no2nd_j2nd[0], reverse_one_hot(data_train_ll_penul_no2nd_j2nd[1][0])-1)
data_gp_test = (data_test_ll_penul_no2nd_j2nd[0], reverse_one_hot(data_test_ll_penul_no2nd_j2nd[1][0])-1)

lengthscales_input = 0.1
data_gp = data_gp_train
white_variance = 0.01
kernel_func = 4
num_inducing = 250
data_test = data_gp_test
num_it = 4000

lengthscales = tf.convert_to_tensor([lengthscales_input] * data_gp[0].shape[1], dtype=default_float())
kernel = kernel_func_map[kernel_func](lengthscales=lengthscales) # (kernel_func_map[kernel_func](lengthscales=lengthscales)  + gpflow.kernels.White(variance=white_variance)) if white_variance > 0.0 else kernel_func_map[kernel_func](lengthscales=lengthscales)
invlink = gpflow.likelihoods.RobustMax(np.max(data_test[1])+1)  # Robustmax inverse link function
likelihood = gpflow.likelihoods.MultiClass(np.max(data_test[1])+1, invlink=invlink)  # Multiclass likelihood
M = num_inducing # Number of inducing locations
Z = data_gp[0][sorted(random.sample(range(0,data_gp[0].shape[0]),M)),:].copy()
mGP = gpflow.models.SVGP(
    kernel=kernel,
    likelihood=likelihood,
    inducing_variable=Z,
    num_latent_gps=np.max(data_test[1])+1,
    whiten=False,
    q_diag=False
)
# Only train the variational parameters
# set_trainable(mGP.kernel.kernels[1].variance, True)
set_trainable(mGP.inducing_variable, False)
opt = gpflow.optimizers.Scipy()
opt_logs = opt.minimize(
    mGP.training_loss_closure(data_gp), mGP.trainable_variables,
    options=dict(maxiter=ci_niter(num_it))
)

(Y_test_GP_mean, Y_test_GP_variance) = mGP.predict_y(data_gp[0])
Y_test_GP_mean = Y_test_GP_mean.numpy()
Y_test_pred = []
for i in range(data_gp[0].shape[0]):
    Y_test_pred.append(np.argmax(Y_test_GP_mean[i,]))
diffs = Y_test_pred - data_gp[1]
Y_test_pred = np.array(Y_test_pred)

    # test_acc = sum(1 for x in diffs if x == 0) / data_gp[0].shape[0]
    # return test_acc, Y_test_pred, (Y_test_GP_mean, Y_test_GP_variance)
import matplotlib.pyplot as plt
plt.plot(Y_test_pred)

import seaborn as sns
# resultsC = [ get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 250, 4000),  get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 100, 4000) ] 

# np.max(np.array(Y_test_pred))
Y_test_pred = np.array(Y_test_pred)
print(data_gp[1][69:79])
print(Y_test_pred[69:79])
print(data_gp[1][2469:2479])
print(Y_test_pred[2469:2479])
# ok tough...
print(np.sum(Y_test_pred[Y_test_pred!=0] - data_gp[1][Y_test_pred!=0]))
print(Y_test_pred[Y_test_pred!=0].shape)
print(Y_test_pred.shape)

In [ ]:
def get_gp_varls_oos(lengthscales_input, white_variance, kernel_func, num_inducing=100, num_it=10000):
  try:
    lengthscales = tf.convert_to_tensor( np.random.uniform(lengthscales_input[0],lengthscales_input[1], size= data_gp_train[0].shape[1]).tolist(), dtype=default_float())
    kernel = (kernel_func_map[kernel_func](lengthscales=lengthscales)  + gpflow.kernels.White(variance=white_variance)) if white_variance > 0.0 else kernel_func_map[kernel_func](lengthscales=lengthscales)
    invlink = gpflow.likelihoods.RobustMax(data_test_ll_no2nd_j2nd[1][0].shape[1])  # Robustmax inverse link function
    likelihood = gpflow.likelihoods.MultiClass(data_test_ll_no2nd_j2nd[1][0].shape[1], invlink=invlink)  # Multiclass likelihood
    M = num_inducing # Number of inducing locations
    Z = data_gp_train[0][sorted(random.sample(range(0,data_gp_train[0].shape[0]),M)),:].copy()
    mGP = gpflow.models.SVGP(
        kernel=kernel,
        likelihood=likelihood,
        inducing_variable=Z,
        num_latent_gps=data_test_ll_no2nd_j2nd[1][0].shape[1],
        whiten=True,
        q_diag=False,
    )
    # Only train the variational parameters
    # set_trainable(mGP.kernel.kernels[1].variance, True)
    set_trainable(mGP.inducing_variable, False)
    opt = gpflow.optimizers.Scipy()
    opt_logs = opt.minimize(
        mGP.training_loss_closure(data_gp_train), mGP.trainable_variables,
        options=dict(maxiter=ci_niter(num_it))
    )
    test_acc, Y_test_pred, (gp_mean, gp_var) = get_gp_acc(mGP, data_gp_test)
    print(test_acc, lengthscales_input, white_variance, kernel_func_map[kernel_func], num_inducing)  
    return test_acc, lengthscales_input, white_variance, kernel_func, num_inducing, Y_test_pred, mGP
  except :
     return -1.0, lengthscales_input, white_variance, kernel_func, num_inducing, None, mGP


In [ ]:
# data_gp_train = (data_train_ll_no2nd_j2nd[0], reverse_one_hot(data_train_ll_no2nd_j2nd[1][0])-1)
# data_gp_test = (data_test_ll_no2nd_j2nd[0], reverse_one_hot(data_test_ll_no2nd_j2nd[1][0])-1)
# data_gp_train = (data_train_ll_no2nd_j2nd_rNo2nd[0], reverse_one_hot(data_train_ll_no2nd_j2nd_rNo2nd[1][0])-1)
# data_gp_test = (data_test_ll_no2nd_j2nd_rNo2nd[0], reverse_one_hot(data_test_ll_no2nd_j2nd_rNo2nd[1][0])-1)

np.max(data_gp_test[1])+1

12

In [ ]:
# just running one
data_gp_train = (data_train_ll_no2nd_j2nd[0], reverse_one_hot(data_train_ll_no2nd_j2nd[1][0])-1)
data_gp_test = (data_test_ll_no2nd_j2nd[0], reverse_one_hot(data_test_ll_no2nd_j2nd[1][0])-1)
resultsA = [ get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 250, 4000),  get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 100, 4000) ] 

data_gp_train = (data_train_ll_no2nd_j2nd_rNo2nd[0], reverse_one_hot(data_train_ll_no2nd_j2nd_rNo2nd[1][0])-1)
data_gp_test = (data_test_ll_no2nd_j2nd_rNo2nd[0], reverse_one_hot(data_test_ll_no2nd_j2nd_rNo2nd[1][0])-1)                                                                                                    
resultsB = [ get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 250, 4000),  get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 100, 4000) ] 


data_gp_train = (data_train_ll_penul_no2nd_j2nd[0], reverse_one_hot(data_train_ll_penul_no2nd_j2nd[1][0])-1)
data_gp_test = (data_test_ll_penul_no2nd_j2nd[0], reverse_one_hot(data_test_ll_penul_no2nd_j2nd[1][0])-1)

resultsC = [ get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 250, 4000),  get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 100, 4000) ] 



0.9638694638694638 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9603729603729604 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 100
0.9568764568764568 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.9557109557109557 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 100
0.08158508158508158 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250
0.08158508158508158 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 100


In [ ]:
lengthscales = tf.convert_to_tensor([0.1] * data_gp_train[0].shape[1], dtype=default_float())
kernel = (kernel_func_map[4](lengthscales=lengthscales)  + gpflow.kernels.White(variance=0.01))
invlink = gpflow.likelihoods.RobustMax(np.max(data_gp_test[1])+1)  # Robustmax inverse link function
likelihood = gpflow.likelihoods.MultiClass(np.max(data_gp_test[1])+1, invlink=invlink)  # Multiclass likelihood
M = 250 # Number of inducing locations
Z = data_gp_train[0][sorted(random.sample(range(0,data_gp_train[0].shape[0]),M)),:].copy()
mGP = gpflow.models.SVGP(
    kernel=kernel,
    likelihood=likelihood,
    inducing_variable=Z,
    num_latent_gps=np.max(data_gp_test[1])+1,
    whiten=True,
    q_diag=False,
)
set_trainable(mGP.inducing_variable, False)
opt = gpflow.optimizers.Scipy()
opt_logs = opt.minimize(
    mGP.training_loss_closure(data_gp_train), mGP.trainable_variables,
    options=dict(maxiter=ci_niter(4000))
)
test_acc, Y_test_pred, (gp_mean, gp_var) = get_gp_acc(mGP, data_gp_test)
print(test_acc, 0.1, 0.01, kernel_func_map[4], 250)  

(Y_test_GP_mean, Y_test_GP_variance) = mGP.predict_y(data_gp_test[0])
Y_test_GP_mean = Y_test_GP_mean.numpy()
Y_test_pred = []
for i in range(data_gp_test[0].shape[0]):
    Y_test_pred.append(np.argmax(Y_test_GP_mean[i,]))

diffs = Y_test_pred - data_gp_test[1]
# test_acc = sum(1 for x in diffs if x == 0) / data_gp_test[0].shape[0]
# return test_acc, Y_test_pred, (Y_test_GP_mean, Y_test_GP_variance)



0.08158508158508158 0.1 0.01 <class 'gpflow.kernels.stationaries.Matern32'> 250


In [ ]:
# np.unique(Y_test_pred)
data_gp_test[0]

array([[-2.28005457,  1.18560743, -1.87473881, ...,  2.22458434,
         0.18365669, -1.73513246],
       [-1.85952389, -1.43467045,  0.38176835, ...,  0.4277342 ,
        -1.3096354 ,  2.12281489],
       [-1.87429988, -0.03654051, -0.12345496, ...,  0.37061286,
        -0.21829401,  2.04799533],
       ...,
       [-1.93701923, -2.35466337,  1.07926047, ...,  0.88620865,
        -3.14220428,  0.96247303],
       [-1.42820859, -1.61642849, -0.89153689, ..., -0.22308743,
        -2.13899112, -2.12054086],
       [-1.37312782, -1.21590495,  1.60235393, ..., -0.40148723,
        -2.90022206, -1.19593   ]])

In [ ]:











data_gp_train = (data_train_ll_penul_no2nd_j2nd_rNo2nd[0], reverse_one_hot(data_train_ll_penul_no2nd_j2nd_rNo2nd[1][0])-1)
data_gp_test = (data_test_ll_penul_no2nd_j2nd_rNo2nd[0], reverse_one_hot(data_test_ll_penul_no2nd_j2nd_rNo2nd[1][0])-1)

resultsD = [ get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 250, 4000),  get_gp_oos(data_gp_train, data_gp_test, 0.1,0.01,4, 100, 4000) ] 


In [ ]:
    lengthscales = tf.convert_to_tensor([lengthscales_input] * data_gp[0].shape[1], dtype=default_float())
    kernel = (kernel_func_map[kernel_func](lengthscales=lengthscales)  + gpflow.kernels.White(variance=white_variance)) if white_variance > 0.0 else kernel_func_map[kernel_func](lengthscales=lengthscales)
    invlink = gpflow.likelihoods.RobustMax(np.max(data_test[1])+1)  # Robustmax inverse link function
    likelihood = gpflow.likelihoods.MultiClass(np.max(data_test[1])+1, invlink=invlink)  # Multiclass likelihood
    M = num_inducing # Number of inducing locations
    Z = data_gp[0][sorted(random.sample(range(0,data_gp[0].shape[0]),M)),:].copy()
    mGP = gpflow.models.SVGP(
        kernel=kernel,
        likelihood=likelihood,
        inducing_variable=Z,
        num_latent_gps=np.max(data_test[1])+1,
        whiten=True,
        q_diag=False,
    )
    # Only train the variational parameters
    # set_trainable(mGP.kernel.kernels[1].variance, True)
    set_trainable(mGP.inducing_variable, False)
    opt = gpflow.optimizers.Scipy()
    opt_logs = opt.minimize(
        mGP.training_loss_closure(data_gp), mGP.trainable_variables,
        options=dict(maxiter=ci_niter(num_it))
    )
    test_acc, Y_test_pred, (gp_mean, gp_var) = get_gp_acc(mGP, data_test)
    print(test_acc, lengthscales_input, white_variance, kernel_func_map[kernel_func], num_inducing)  


In [ ]:
res2 = [get_gp_varls_oos( [ 0.08, 0.09] ,0.001,4, 40, 4000) for x in range(10)]

0.7668997668997669 [0.08, 0.09] 0.001 <class 'gpflow.kernels.stationaries.Matern32'> 40
0.9067599067599068 [0.08, 0.09] 0.001 <class 'gpflow.kernels.stationaries.Matern32'> 40
0.9615384615384616 [0.08, 0.09] 0.001 <class 'gpflow.kernels.stationaries.Matern32'> 40
0.8041958041958042 [0.08, 0.09] 0.001 <class 'gpflow.kernels.stationaries.Matern32'> 40
0.6655011655011654 [0.08, 0.09] 0.001 <class 'gpflow.kernels.stationaries.Matern32'> 40
0.9067599067599068 [0.08, 0.09] 0.001 <class 'gpflow.kernels.stationaries.Matern32'> 40
0.9393939393939394 [0.08, 0.09] 0.001 <class 'gpflow.kernels.stationaries.Matern32'> 40
0.8881118881118881 [0.08, 0.09] 0.001 <class 'gpflow.kernels.stationaries.Matern32'> 40
0.9382284382284383 [0.08, 0.09] 0.001 <class 'gpflow.kernels.stationaries.Matern32'> 40
0.9300699300699301 [0.08, 0.09] 0.001 <class 'gpflow.kernels.stationaries.Matern32'> 40


In [ ]:
# running the whole combination set
# results = np.apply_along_axis(lambda x: get_gp_oos(x[0],x[1],int(x[2]), int(x[3])) , 1, combs) 

